In [ ]:
!git clone https://github.com/SEGAPE/relatorio_prefeitos_webapp.git


Cloning into 'relatorio_prefeitos_webapp'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 36 (delta 10), reused 19 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 57.97 KiB | 5.27 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
!ls relatorio_prefeitos_webapp
!ls relatorio_prefeitos_webapp/src
!ls relatorio_prefeitos_webapp/src/static


LICENSE  README.md  requirements.txt  src
app.py	static
municipios.csv


In [ ]:
# 1. Clonar
!git clone https://github.com/SEGAPE/relatorio_prefeitos_webapp.git

# 2. Caminho do CSV no repositório clonado
CSV_PATH = "relatorio_prefeitos_webapp/src/static/municipios.csv"

# 3. O resto do código de scraping
import os
import pandas as pd
import urllib.parse
import requests
from tqdm.notebook import tqdm

OUTPUT_DIR = "relatorios_prefeitos"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH, dtype={"id_municipio": str})
print(f"Total de municípios: {len(df)}")

def montar_url(codigo_ibge, nome, uf):
    nome_url = urllib.parse.quote(nome, safe="")
    return f"https://storage.googleapis.com/br-mec-privado-relatorio-prefeitos/relatorio_prefeitos/{uf}/{codigo_ibge}_{nome_url}_{uf}.pdf.pdf"

def baixar_relatorio(row):
    codigo_ibge = row["id_municipio"]
    nome = row["nome"]
    uf = row["sigla_uf"]
    url = montar_url(codigo_ibge, nome, uf)
    nome_arquivo = f"{codigo_ibge}_{nome}_{uf}.pdf".replace(" ", "_")
    caminho = os.path.join(OUTPUT_DIR, nome_arquivo)
    try:
        resp = requests.head(url, timeout=5)
        if resp.status_code == 200:
            resp2 = requests.get(url, timeout=10)
            with open(caminho, "wb") as f:
                f.write(resp2.content)
            return {"municipio": nome, "uf": uf, "codigo_ibge": codigo_ibge, "status": "OK", "url": url}
        else:
            return {"municipio": nome, "uf": uf, "codigo_ibge": codigo_ibge, "status": f"NOT_FOUND ({resp.status_code})", "url": url}
    except Exception as e:
        return {"municipio": nome, "uf": uf, "codigo_ibge": codigo_ibge, "status": f"ERROR ({e})", "url": url}

resultados = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    resultados.append(baixar_relatorio(row))

df_res = pd.DataFrame(resultados)
df_res.to_csv("resultado_relatorios.csv", index=False)
print("Feito! CSV de resultado gerado: resultado_relatorios.csv")


fatal: destination path 'relatorio_prefeitos_webapp' already exists and is not an empty directory.
Total de municípios: 5570


  0%|          | 0/5570 [00:00<?, ?it/s]

Feito! CSV de resultado gerado: resultado_relatorios.csv


In [ ]:
# ============================================================
# 📥 SCRAPING DE RELATÓRIOS DE PREFEITOS COM SALVAMENTO INCREMENTAL
# ============================================================

# 1️⃣ Clonar o repositório
!git clone https://github.com/SEGAPE/relatorio_prefeitos_webapp.git

# 2️⃣ Conectar ao Google Drive logo no início
from google.colab import drive
drive.mount('/content/drive')

# 3️⃣ Imports
import os
import pandas as pd
import urllib.parse
import requests
from tqdm.notebook import tqdm
import shutil

# 4️⃣ Configurações
CSV_PATH = "relatorio_prefeitos_webapp/src/static/municipios.csv"
OUTPUT_DIR_LOCAL = "/content/relatorios_prefeitos"
DRIVE_FOLDER = "/content/drive/MyDrive/relatorios_prefeitos"

# Criar pastas
os.makedirs(OUTPUT_DIR_LOCAL, exist_ok=True)
os.makedirs(DRIVE_FOLDER, exist_ok=True)

# 5️⃣ Carregar CSV de municípios
df = pd.read_csv(CSV_PATH, dtype={"id_municipio": str})
print(f"📊 Total de municípios: {len(df)}")

# 6️⃣ Verificar se já existe um log de progresso
LOG_FILE = os.path.join(DRIVE_FOLDER, "resultado_relatorios.csv")
if os.path.exists(LOG_FILE):
    df_log = pd.read_csv(LOG_FILE)
    codigos_baixados = set(df_log[df_log['status'] == 'OK']['codigo_ibge'].astype(str))
    print(f"✅ Encontrados {len(codigos_baixados)} relatórios já baixados")
else:
    codigos_baixados = set()
    df_log = pd.DataFrame(columns=["municipio", "uf", "codigo_ibge", "status", "url"])

# 7️⃣ Função para montar URL
def montar_url(codigo_ibge, nome, uf):
    nome_url = urllib.parse.quote(nome, safe="")
    return f"https://storage.googleapis.com/br-mec-privado-relatorio-prefeitos/relatorio_prefeitos/{uf}/{codigo_ibge}_{nome_url}_{uf}.pdf.pdf"

# 8️⃣ Função para baixar e salvar diretamente no Drive
def baixar_e_salvar(row):
    codigo_ibge = row["id_municipio"]
    nome = row["nome"]
    uf = row["sigla_uf"]

    # Pular se já foi baixado
    if str(codigo_ibge) in codigos_baixados:
        return None

    url = montar_url(codigo_ibge, nome, uf)
    nome_arquivo = f"{codigo_ibge}_{nome}_{uf}.pdf".replace(" ", "_")
    caminho_drive = os.path.join(DRIVE_FOLDER, nome_arquivo)

    try:
        resp = requests.head(url, timeout=5)
        if resp.status_code == 200:
            resp2 = requests.get(url, timeout=15)
            # Salvar direto no Drive
            with open(caminho_drive, "wb") as f:
                f.write(resp2.content)
            return {
                "municipio": nome,
                "uf": uf,
                "codigo_ibge": codigo_ibge,
                "status": "OK",
                "url": url
            }
        else:
            return {
                "municipio": nome,
                "uf": uf,
                "codigo_ibge": codigo_ibge,
                "status": f"NOT_FOUND ({resp.status_code})",
                "url": url
            }
    except Exception as e:
        return {
            "municipio": nome,
            "uf": uf,
            "codigo_ibge": codigo_ibge,
            "status": f"ERROR ({str(e)[:50]})",
            "url": url
        }

# 9️⃣ Loop de download com salvamento incremental
print("🚀 Iniciando downloads...")
resultados_novos = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    resultado = baixar_e_salvar(row)

    if resultado:
        resultados_novos.append(resultado)

        # Salvar log a cada 10 downloads
        if len(resultados_novos) % 10 == 0:
            df_novos = pd.DataFrame(resultados_novos)
            df_atualizado = pd.concat([df_log, df_novos], ignore_index=True)
            df_atualizado.to_csv(LOG_FILE, index=False)

# 🔟 Salvar log final
if resultados_novos:
    df_novos = pd.DataFrame(resultados_novos)
    df_atualizado = pd.concat([df_log, df_novos], ignore_index=True)
    df_atualizado.to_csv(LOG_FILE, index=False)
    print(f"\n✅ {len(resultados_novos)} novos relatórios baixados!")
else:
    print("\n✅ Nenhum relatório novo para baixar")

# 1️⃣1️⃣ Estatísticas finais
df_final = pd.read_csv(LOG_FILE)
print("\n📈 ESTATÍSTICAS FINAIS:")
print(f"   • Total processados: {len(df_final)}")
print(f"   • Sucesso (OK): {len(df_final[df_final['status'] == 'OK'])}")
print(f"   • Não encontrados: {len(df_final[df_final['status'].str.contains('NOT_FOUND')])}")
print(f"   • Erros: {len(df_final[df_final['status'].str.contains('ERROR')])}")
print(f"\n💾 Arquivos salvos em: {DRIVE_FOLDER}")

Cloning into 'relatorio_prefeitos_webapp'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 36 (delta 10), reused 19 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 57.97 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Mounted at /content/drive
📊 Total de municípios: 5570
✅ Encontrados 150 relatórios já baixados
🚀 Iniciando downloads...


  0%|          | 0/5570 [00:00<?, ?it/s]


✅ 5420 novos relatórios baixados!

📈 ESTATÍSTICAS FINAIS:
   • Total processados: 5570
   • Sucesso (OK): 5570
   • Não encontrados: 0
   • Erros: 0

💾 Arquivos salvos em: /content/drive/MyDrive/relatorios_prefeitos


In [ ]:
pip install PyPDF2

In [ ]:
import os
import re
import PyPDF2
import pandas as pd
from datetime import datetime
import pytz
from google.colab import drive
from multiprocessing import Pool, cpu_count
import json
import hashlib
from tqdm.auto import tqdm

# ============================================================
# 📁 CONFIGURAÇÃO
# ============================================================

drive.mount('/content/drive')

PDF_DIR = "/content/drive/MyDrive/relatorios_prefeitos"
OUTPUT_FILE = "/content/drive/MyDrive/analise_tempo_integral_completa.csv"
CACHE_DIR = "/content/drive/MyDrive/cache_pdfs"
SAVE_EVERY = 100
NUM_PROCESSOS = cpu_count()

# 🆕 FILTRO POR ESTADO (deixe vazio para processar todos)
# Exemplos: ['SP'], ['RJ', 'MG'], [] para todos
FILTRAR_ESTADOS = []  # Deixe vazio [] para processar TODOS

TZ_BRASILIA = pytz.timezone('America/Sao_Paulo')

os.makedirs(CACHE_DIR, exist_ok=True)

print("=" * 80)
print("🚀 EXTRAÇÃO - OTIMIZADA (PÁGINAS 5-10 + FILTRO ESTADO)")
print("=" * 80)
print(f"⏰ Início: {datetime.now(TZ_BRASILIA).strftime('%H:%M:%S')}")
print(f"🔥 Processos: {NUM_PROCESSOS}")
print(f"💾 Salva a cada: {SAVE_EVERY} PDFs")
print(f"📦 Cache: {CACHE_DIR}")
if FILTRAR_ESTADOS:
    print(f"🗺️  Filtro: Apenas estados {', '.join(FILTRAR_ESTADOS)}")
else:
    print(f"🗺️  Filtro: TODOS os estados")
print()

# ============================================================
# 🔧 FUNÇÕES
# ============================================================

def extrair_info_nome_arquivo(filename):
    match = re.match(r'(\d+)_(.+)_([A-Z]{2})\.pdf', filename)
    if match:
        return {
            'codigo_ibge': match.group(1),
            'municipio': match.group(2).replace('_', ' '),
            'uf': match.group(3)
        }
    return None

def limpar_numero(texto):
    if not texto:
        return 0.0
    texto = texto.strip()
    texto = re.sub(r'R\$\s*', '', texto)
    texto = texto.replace('.', '').replace(',', '.')
    try:
        return float(texto)
    except:
        return 0.0

def get_cache_path(filename):
    cache_name = hashlib.md5(filename.encode()).hexdigest() + '.json'
    return os.path.join(CACHE_DIR, cache_name)

def carregar_cache(filename):
    cache_path = get_cache_path(filename)
    if os.path.exists(cache_path):
        try:
            with open(cache_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return None
    return None

def salvar_cache(filename, dados):
    cache_path = get_cache_path(filename)
    try:
        with open(cache_path, 'w', encoding='utf-8') as f:
            json.dump(dados, f, ensure_ascii=False)
    except:
        pass

def processar_pdf_completo(arquivo):
    """Processa um PDF - OTIMIZADO para ler só páginas 5-10"""
    try:
        # 1. Tentar cache
        cache_dados = carregar_cache(arquivo)
        if cache_dados is not None:
            cache_dados['from_cache'] = True
            return cache_dados

        # 2. Extrair info do nome
        info = extrair_info_nome_arquivo(arquivo)
        if not info:
            return None

        # 3. Ler PDF - OTIMIZAÇÃO: SÓ PÁGINAS 5-10
        caminho = os.path.join(PDF_DIR, arquivo)
        with open(caminho, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            texto = ""

            # 🚀 OTIMIZAÇÃO: Ler apenas páginas 5-10 (índices 4-9)
            num_paginas = len(reader.pages)
            inicio_pag = min(4, num_paginas)  # Página 5 (índice 4)
            fim_pag = min(10, num_paginas)    # Até página 10 (índice 9)

            for page in reader.pages[inicio_pag:fim_pag]:
                texto += page.extract_text() + " "

        texto = texto.replace('\n', ' ').replace('  ', ' ')

        # 4. Extrair dados
        dados = {
            'saldo_conta': 0.0,
            'ciclo1_valor_pago': 0.0,
            'ciclo1_matriculas_pactuadas': 0,
            'ciclo1_matriculas_declaradas': 0,
            'ciclo2_valor_estimado': 0.0,
            'ciclo2_matriculas_pactuadas': 0,
            'tem_dados': False
        }

        match = re.search(r'R\$\s*([\d.,]+)\s+SITUAÇÃO\s+DO\s+MUNICÍPIO', texto, re.IGNORECASE)
        if match:
            dados['saldo_conta'] = limpar_numero(match.group(1))

        match = re.search(r'1º\s+Ciclo.*?(\d+)\s+Matrículas\s+Pactuadas', texto, re.IGNORECASE | re.DOTALL)
        if match:
            dados['ciclo1_matriculas_pactuadas'] = int(match.group(1))

        match = re.search(r'(\d+)\s+Matrículas\s+Declaradas', texto, re.IGNORECASE)
        if match:
            dados['ciclo1_matriculas_declaradas'] = int(match.group(1))

        match = re.search(r'R\$\s*([\d.,]+)\s+Valor\s+Pago', texto, re.IGNORECASE)
        if match:
            dados['ciclo1_valor_pago'] = limpar_numero(match.group(1))
            dados['tem_dados'] = True

        match = re.search(r'2º\s+Ciclo.*?(\d+)\s+Matrículas\s+Pactuadas', texto, re.IGNORECASE | re.DOTALL)
        if match:
            dados['ciclo2_matriculas_pactuadas'] = int(match.group(1))

        match = re.search(r'R\$\s*([\d.,]+)\s+Valor\s+Estimado', texto, re.IGNORECASE)
        if match:
            dados['ciclo2_valor_estimado'] = limpar_numero(match.group(1))
            dados['tem_dados'] = True

        # 5. Montar resultado
        resultado = {
            'codigo_ibge': info['codigo_ibge'],
            'municipio': info['municipio'],
            'uf': info['uf'],
            'saldo_conta': dados['saldo_conta'],
            'ciclo1_valor_pago': dados['ciclo1_valor_pago'],
            'ciclo1_matriculas_pactuadas': dados['ciclo1_matriculas_pactuadas'],
            'ciclo1_matriculas_declaradas': dados['ciclo1_matriculas_declaradas'],
            'ciclo2_valor_estimado': dados['ciclo2_valor_estimado'],
            'ciclo2_matriculas_pactuadas': dados['ciclo2_matriculas_pactuadas'],
            'valor_total': dados['ciclo1_valor_pago'] + dados['ciclo2_valor_estimado'],
            'tem_dados': dados['tem_dados'],
            'from_cache': False
        }

        # 6. Salvar cache
        salvar_cache(arquivo, resultado)

        return resultado

    except Exception as e:
        return None

# ============================================================
# 🔄 PROCESSAMENTO
# ============================================================

# Listar todos os PDFs
todos_pdfs = sorted([f for f in os.listdir(PDF_DIR) if f.endswith('.pdf')])

# 🆕 FILTRAR POR ESTADO se configurado
if FILTRAR_ESTADOS:
    pdf_files = []
    for pdf in todos_pdfs:
        info = extrair_info_nome_arquivo(pdf)
        if info and info['uf'] in FILTRAR_ESTADOS:
            pdf_files.append(pdf)
    print(f"📊 Total de PDFs (filtrados): {len(pdf_files)} de {len(todos_pdfs)}")
else:
    pdf_files = todos_pdfs
    print(f"📊 Total de PDFs: {len(pdf_files)}")

total_arquivos = len(pdf_files)

print(f"📁 Diretório: {PDF_DIR}")

# Verificar cache existente
cache_existente = sum(1 for f in pdf_files if carregar_cache(f) is not None)
if cache_existente > 0:
    print(f"📦 PDFs em cache: {cache_existente} ({cache_existente/total_arquivos*100:.1f}%)")
    usar_cache = input("♻️  Usar cache existente? (s/n): ").lower().strip()
    if usar_cache != 's':
        print("🧹 Limpando cache...")
        import shutil
        if os.path.exists(CACHE_DIR):
            shutil.rmtree(CACHE_DIR)
        os.makedirs(CACHE_DIR, exist_ok=True)
else:
    print("📦 Nenhum cache encontrado")

print("🚀 Iniciando processamento paralelo...\n")

resultados = []
inicio = datetime.now(TZ_BRASILIA)

total_com_dados = 0
total_valor = 0.0
total_cache = 0
total_novos = 0

# PROCESSAMENTO COM IMAP_UNORDERED
with Pool(NUM_PROCESSOS) as pool:
    with tqdm(total=total_arquivos, desc="📊 Processando", unit="PDF") as pbar:

        for resultado in pool.imap_unordered(processar_pdf_completo, pdf_files, chunksize=5):

            if resultado is not None:
                # Atualizar contadores
                if resultado.get('from_cache', False):
                    total_cache += 1
                else:
                    total_novos += 1

                resultado.pop('from_cache', None)

                if resultado['tem_dados']:
                    total_com_dados += 1
                    total_valor += resultado['valor_total']

                resultados.append(resultado)

                # Salvar periodicamente
                if len(resultados) % SAVE_EVERY == 0:
                    df_temp = pd.DataFrame(resultados)
                    df_temp.to_csv(OUTPUT_FILE, index=False)

                # Atualizar barra
                pbar.update(1)
                pbar.set_postfix({
                    'Dados': total_com_dados,
                    'Valor': f'R${total_valor/1000000:.1f}M' if total_valor > 0 else 'R$0',
                    'Cache': total_cache,
                    'Novos': total_novos
                })

# Salvar final
print("\n💾 Salvando arquivo final...")
df_final = pd.DataFrame(resultados)
df_final.to_csv(OUTPUT_FILE, index=False)

fim = datetime.now(TZ_BRASILIA)
duracao = (fim - inicio).total_seconds() / 60

print("\n" + "=" * 80)
print("✅ PROCESSAMENTO CONCLUÍDO!")
print("=" * 80)
print(f"⏰ Término: {fim.strftime('%H:%M:%S')}")
print(f"⏱️  Duração total: {duracao:.1f} minutos")
print(f"⚡ Velocidade média: {len(df_final)/duracao:.1f} PDFs/minuto")
print(f"🔥 Processos: {NUM_PROCESSOS}")
print(f"📦 Do cache: {total_cache} ({total_cache/len(df_final)*100:.1f}%)")
print(f"🆕 Processados: {total_novos} ({total_novos/len(df_final)*100:.1f}%)\n")

print("📊 ESTATÍSTICAS GERAIS:")
print(f"   • Total de municípios: {len(df_final)}")
print(f"   • Com recursos: {len(df_final[df_final['tem_dados']])} ({len(df_final[df_final['tem_dados']])/len(df_final)*100:.1f}%)")
print(f"   • Sem recursos: {len(df_final[~df_final['tem_dados']])} ({len(df_final[~df_final['tem_dados']])/len(df_final)*100:.1f}%)\n")

df_com_dados = df_final[df_final['tem_dados']]

if len(df_com_dados) > 0:
    print("💰 VALORES TOTAIS:")
    print(f"   • Saldo em conta: R$ {df_com_dados['saldo_conta'].sum():,.2f}")
    print(f"   • Ciclo 1 (pago): R$ {df_com_dados['ciclo1_valor_pago'].sum():,.2f}")
    print(f"   • Ciclo 2 (estimado): R$ {df_com_dados['ciclo2_valor_estimado'].sum():,.2f}")
    print(f"   • TOTAL GERAL: R$ {df_com_dados['valor_total'].sum():,.2f}\n")

    print("📈 ESTATÍSTICAS DE VALORES:")
    print(f"   • Média por município: R$ {df_com_dados['valor_total'].mean():,.2f}")
    print(f"   • Mediana: R$ {df_com_dados['valor_total'].median():,.2f}")
    print(f"   • Maior valor: R$ {df_com_dados['valor_total'].max():,.2f}")
    print(f"   • Menor valor: R$ {df_com_dados[df_com_dados['valor_total'] > 0]['valor_total'].min():,.2f}\n")

    print("👥 ESTATÍSTICAS DE MATRÍCULAS:")
    total_c1_pac = df_com_dados['ciclo1_matriculas_pactuadas'].sum()
    total_c1_dec = df_com_dados['ciclo1_matriculas_declaradas'].sum()
    total_c2_pac = df_com_dados['ciclo2_matriculas_pactuadas'].sum()
    print(f"   • Ciclo 1 - Pactuadas: {total_c1_pac:,}")
    print(f"   • Ciclo 1 - Declaradas: {total_c1_dec:,}")
    print(f"   • Ciclo 2 - Pactuadas: {total_c2_pac:,}\n")

    print("🏆 TOP 10 MUNICÍPIOS POR VALOR TOTAL:")
    top10 = df_com_dados.nlargest(10, 'valor_total')[['municipio', 'uf', 'valor_total']]
    for idx, row in top10.iterrows():
        print(f"   {row['municipio']} ({row['uf']}): R$ {row['valor_total']:,.2f}")

    if FILTRAR_ESTADOS:
        print(f"\n🗺️  ESTATÍSTICAS POR ESTADO:")
        for estado in FILTRAR_ESTADOS:
            df_estado = df_com_dados[df_com_dados['uf'] == estado]
            if len(df_estado) > 0:
                print(f"   • {estado}: {len(df_estado)} municípios - R$ {df_estado['valor_total'].sum():,.2f}")

print(f"\n💾 Arquivo salvo: {OUTPUT_FILE}")
print(f"📦 Cache salvo: {CACHE_DIR}")
print("=" * 80)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 EXTRAÇÃO - OTIMIZADA (PÁGINAS 5-10 + FILTRO ESTADO)
⏰ Início: 10:50:57
🔥 Processos: 2
💾 Salva a cada: 100 PDFs
📦 Cache: /content/drive/MyDrive/cache_pdfs
🗺️  Filtro: TODOS os estados

📊 Total de PDFs: 5570
📁 Diretório: /content/drive/MyDrive/relatorios_prefeitos
📦 PDFs em cache: 94 (1.7%)
♻️  Usar cache existente? (s/n): s
🚀 Iniciando processamento paralelo...



📊 Processando:   0%|          | 0/5570 [00:00<?, ?PDF/s]


💾 Salvando arquivo final...

✅ PROCESSAMENTO CONCLUÍDO!
⏰ Término: 13:14:54
⏱️  Duração total: 143.9 minutos
⚡ Velocidade média: 38.7 PDFs/minuto
🔥 Processos: 2
📦 Do cache: 94 (1.7%)
🆕 Processados: 5476 (98.3%)

📊 ESTATÍSTICAS GERAIS:
   • Total de municípios: 5570
   • Com recursos: 5569 (100.0%)
   • Sem recursos: 1 (0.0%)

💰 VALORES TOTAIS:
   • Saldo em conta: R$ 1,598,189,357.51
   • Ciclo 1 (pago): R$ 3,068,255,042.85
   • Ciclo 2 (estimado): R$ 2,720,695,204.39
   • TOTAL GERAL: R$ 5,788,950,247.24

📈 ESTATÍSTICAS DE VALORES:
   • Média por município: R$ 1,039,495.47
   • Mediana: R$ 406,373.40
   • Maior valor: R$ 99,734,754.35
   • Menor valor: R$ 7,400.00

👥 ESTATÍSTICAS DE MATRÍCULAS:
   • Ciclo 1 - Pactuadas: 443,321
   • Ciclo 1 - Declaradas: 401,026
   • Ciclo 2 - Pactuadas: 443,321

🏆 TOP 10 MUNICÍPIOS POR VALOR TOTAL:
   Manaus (AM): R$ 99,734,754.35
   São Paulo (SP): R$ 73,313,039.56
   Brasília (DF): R$ 55,494,050.00
   Rio de Janeiro (RJ): R$ 45,291,708.54
   Duq